In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
# import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
from os import path
import roamability as rb
import re
import json

downloads = 'C:/Users/balob/Downloads'

In [ ]:
%reset -f

# Examples

### MS SQL request

In [ ]:
sql_srt='SELECT COUNT(*) FROM ACCOUNTS;'
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### DMI Oracle request

In [ ]:
sql_srt = 'SELECT * FROM SPONSOR'
# with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI_TEST') as cnxn:
with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### BSS MySql request

In [ ]:
sql_srt='SELECT MSISDN, VisitedNetworkTadig FROM TAP.GPRS_CALL LIMIT 5'
with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn)
df

### Soap API OCS Portal

In [27]:
# %reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    user = 'api_lab@dialoq.com'
    user = 'juan.perez@altanredes.com' # E10ADC3949BA59ABBE56E057F20F883E
    user = 'api@altanredes.com' # api Altan prod
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    password = 'FAC4EC08B5FC0CCF8145BB79E8F24F47' #Lab
    password =  'E10ADC3949BA59ABBE56E057F20F883E' # lab 'juan.perez@altanredes.com'
    password = '2AF9B1BA42DC5EB01743E6B3759B6E4B'  # prod 'api@altanredes.com'
    api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
    api_link = 'http://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    user = 'api@volnamobile.ru'
    password = '25D55AD283AA400AF464C76D713C07AD'
    password = 'E10ADC3949BA59ABBE56E057F20F883E'
    api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/'
#     api_link = 'https://ocsapi2.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.10:8585/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))

# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934420')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
# client.service.getSubscriberPrePaidPackages('831882', 1)
#client.service.getSubscriberByIMSI('425019613023160')
#client.service.getSubscriberById('209287')
#client.service.getSubscriberByICCID('8997219121000022660')
# client.service.assignSubscriberPrePaidPackage('831882', 1, dt.datetime.strptime('2019-07-07','%Y-%m-%d'))
# client.service.getSubscriberPrePaidPackages('937939', 1)
# client.service.expireSubscriberPrePaidPackage(1848972,60311)
# client.service.getSubscriberPrePaidPackages(1848972,1)
# prices = client.service.getMobilePrices() # prices.destinations[0]
# dent = client.service.getMobilePrices()
# for dest in dent.destinations:
#     if dest.tadigCode == '':
#         print(dest)
# client.service.getPricingPlans()
# client.service.modifySubscriberSpeedLimit(1449606, '256kbps')
# client.service.modifySubscriberSpeedLimit(210000, '1024kbps') # lab
#subscriber_id

Choose connection (prod or lab):prod
You are connected to Reseller: VolnaMobile.    
Available credit for the Reseller: 106 USD.

The Reseller contains the following accounts:
 0    VolnaTestSimCards (accountID: 352942)
 1    Volna_201016_10 (accountID: 353029)


In [ ]:
client.service.

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [358]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/_NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
# any_tags  = ['#reports_bss_usage_mop']
# any_tags = ['#bss_mop']
any_tags  = ['#gy_principle_of_operation']
# any_tags  = ['#ocs_principle_of_operation']
# any_tags  = ['#reports_ocs_usage_mop']
# any_tags  = ['#reports_ocs_subscribers']
# any_tags = ['#epg_principle_of_operation']
# any_tags  = ['#reports_ocs_subscribers']
# any_tags  = ['#reports_dmi_subscribers']
# any_tags  = ['#ss7_principle_of_operation']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\ProtocolsDictionariesDevelopingSolutions.txt
----------------------------------------------------------------------------------------------------
#ss7_principle_of_operation	
#s6a_principle_of_operation
#gy_principle_of_operation
#gx_principle_of_operation
------------------------------
Описание протоколов MAP, Diameter и др.
https://www.developingsolutions.com/products/
https://www.developingsolutions.com/MapDict/Topics/About.htm
https://www.developingsolutions.com/DiaDict/Dictionary/APN-Configuration.html

****************************************************************************************************
c:\W_DATA_ROAM\МЕТОДИКИ\_NOC_M_NEW_CONNECTION\027_OCS_Diameter_Gy_and_Relay\readme.txt
----------------------------------------------------------------------------------------------------
#gy_principle_of_operation
------------------------

### Search in ALARMS by folder names

In [439]:
import os

paths = [
         r'c:/W_DATA_ROAM/ALARMS/',
         r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ_NEW_CONNECTION/',
         r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/',
         r'c:/W_DATA_ROAM_DOC/ПРОЕКТЫ/',
        ]

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('KWTNM')!=-1]

### Search in Allowed Lists and others

In [13]:
def get_df_al():
    """
    Get Allowed lists from DMO Oracle DB
    """
    sql_str = """
SELECT
s.name AS Sponsor, o.plmn_code AS TADIG, al.mo_price AS MOC, al.mb_price
FROM allowed_list_p al, sponsor_p s, oper_p o
WHERE al.sponsor_ref = s.ri
AND al.operator_ref = o.ri
"""
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df_al = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
    df_al.rename({'SPONSOR':'Sponsor', 'MB_PRICE':'3G_ONLY'}, axis=1, inplace=True)
    return df_al

In [215]:
downloads = 'C:/Users/balob/Downloads'
# dmi_allowed_list = 'DMI.dmi_allowed_list_export_Fri_Oct_30_2020.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Wed_Dec_09_2020.csv'
bss_usage = 'bss_usage_2019_08.csv' # c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\BSS_SQL\Report_BSS_Usage_per_PLMN.txt

#Like expression for Allowed List
# df_al=pd.read_csv(join(downloads, dmi_allowed_list))

df_al = get_df_al()

df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*':'S1',
                                                        r'[Pp]4[\s\w-]*|^S2_[\s\w-]*':'S2',
                                                        r'^[Mm][Bb][\s\w-]*|^S4_[\s\w-]*':'S4',
                                                        r'^SMART[\s\w-]*|^S5_[\s\w-]*':'S5',
                                                        r'^S6_[\s\w-]*':'S6',
                                                        r'^S8_[\s\w-]*':'S8'}, regex=True)
df_al.rename({'PLMN code':'TADIG', 'MOC':'MO'}, axis=1, inplace=True)
# df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)

df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['REP_DATE','VPLMN','SPONSOR','MOC','MOC_SMS','DATA']
df_usage = df_usage.pivot_table(values='COST', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]
df_usage.rename({'VPLMN':'TADIG'}, axis=1, inplace=True)
df_usage.columns.name=''

df_al = pd.merge(df_al, df_usage, how='left', on=['SPONSOR','TADIG'])

df_np=pd.read_csv(join(downloads, dmi_netpfx))

In [424]:
sponsor = 'dial' # Use regular expressions if it's needed
tadig = 'SAU'

df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG'])

,Sponsor,TADIG,REP_DATE,MOC,MOC_SMS,DATA,SPONSOR
1474,MB_DIAL01_COMB,SAUVG,NaN,NaN,NaN,NaN,S4
8855,MB_DIAL02_COMB,SAUVG,NaN,NaN,NaN,NaN,S4
1470,MB_DIAL_COMB,SAUVG,NaN,NaN,NaN,NaN,S4


In [ ]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','SPONSOR']].sort_values(by=['Sponsor','TADIG']).Sponsor.unique()

In [440]:
plmn = 'KWTNM'
df1 = df_al.loc[(df_al['TADIG'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','TADIG']].sort_values(by=['Sponsor','TADIG'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,TADIG
1502,S6_DRIM01_ALL,KWTNM


,Operator,Prefix,Number/Range,SSN,MCC,MNC
456,KWTNM,9656,1,7,419,3
457,KWTNM,9656,1,149,419,3
5150,KWTNM,9656961830,0,7,419,3
8404,KWTNM,9656630105,0,149,419,3
8405,KWTNM,9656630106,0,149,419,3
8406,KWTNM,9656630108,0,149,419,3
8407,KWTNM,9656961831,0,149,419,3
8408,KWTNM,9656961832,0,149,419,3
10425,KWTNM,96566301050,0,1,419,3


In [361]:
# GSMA NN
plmn = 'ESTEM'
df_ir21_nn = pd.read_csv(join(downloads, 'Global_Title_Number_Ranges_2020-03-19.csv'), dtype='str',
                         usecols=['Country','Operator','TADIG code','CC','NDC','SN range start','SN range stop'])
df_ir21_nn.loc[df_ir21_nn['TADIG code']==plmn]

,Country,Operator,TADIG code,CC,NDC,SN range start,SN range stop
602,Estonia,Telia Eesti AS,ESTEM,372,50,90000,99999


In [88]:
plmn = 'LSO'
df2 = df_np[(df_np.Operator.str.startswith(plmn, na=False)) &
           (df_np.SSN == 7)] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df2)

,Operator,Prefix,Number/Range,SSN,MCC,MNC
890,LSOVL,2665,1,7,651,1
5160,LSOVL,2665582003,0,7,651,1
5161,LSOVL,2665582000,0,7,651,1


In [360]:
str(df_np[(df_np.Prefix.str.startswith('201222000101',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('37250', na=False)) & (df_np.SSN == 7)]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
180,ESTEM,37250,1,7,248,1
4247,ESTEM,3725097007,0,7,248,1
4248,ESTEM,3725097008,0,7,248,1
5670,ESTEM,3725098011,0,7,248,1
5671,ESTEM,3725098012,0,7,248,1
5672,ESTEM,3725098013,0,7,248,1
5673,ESTEM,3725098014,0,7,248,1
5674,ESTEM,3725098015,0,7,248,1
5675,ESTEM,3725098016,0,7,248,1


### Кодирование / декодирование

In [ ]:
len('0114010301001E01140000000300030501020000000000000000000000000000000000000000000000000000000000'.replace(' ', '')) / 2

In [146]:
hex_int = 0x14
dec_int = 15000
bin_int = 0b11100
# Hex to dec
print('Hex to dec')
print(f'{hex_int:0>16d}')
# Hex to bin
print('Hex to bin')
print(f'{hex_int:0>16b}')
# Dec to hex
print('Dec to hex')
print(f'{dec_int:F>16X}')
# Dec to bin
print('Dec to bin')
print(f'{dec_int:0>16b}')
# Bin to hex
print('Bin to hex')
print(f'{bin_int:F>16X}')
# Bin to dec
print('Bin to dec')
print(f'{bin_int:0>16d}')
'250'

'01010010' # 52

Hex to dec
0000000000000020
Hex to bin
0000000000010100
Dec to hex
FFFFFFFFFFFF3A98
Dec to bin
0011101010011000
Bin to hex
FFFFFFFFFFFFFF1C
Bin to dec
0000000000000028


'01010010'

In [ ]:
7-243-1
4-4-2

f'{bin(7)} - {bin(243)} - {bin(1)}'
#2.61.6

In [ ]:
print(bin(10915))
a = '11111110011001'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

In [2]:
# Кодировка

import pyDes  # pyDes if installed from pip
plaintext = '4E3F0E5A000000010018BC'
key_list = [
    'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1',
    '26B60249EC3D7CCE4AADE73F0D803E470AEA1E2C0279AD5D',
    '9B68C23B919D620B9F8627ECFC8E0BD824197152B4C9CB0E',
    'D04C6E02F201B9E057F6DC45ADD217D7C07ABA5CED76ABF4',
    '67A1C10D23239E190495643C3A233B76D1CE047A9D7D5602',
    '0104BA83E6B3B64FB0207B309B6E40C3EBB7E10B0D751920',
    'DCF410DA0BC4E552F11EFF1FA30B193A5B0FACA7061C310E',
    'CB43F7734A0DFB4AA332E690CE0D31F31CB9A6024804DAF2',
    'A4CDDF91ADD34649AD1956FE90333EF5C3012EBF25D7EB53',
    'FEEA7CA8ECE6DAF856EEE86785F7DFE5AE3FF8978E4D1F5B',
]
# Convert plaintext to bytes
plaintext = bytes.fromhex(plaintext)
# Add padding
# length = 8 - (len(plaintext) % 8)
# plaintext += bytes([length]) * length
print(f'original plaintext: {plaintext.hex().upper()}')
iv = bytes.fromhex('0000000000000000')
# Make the encryption
for key in key_list:
    # Convert key to bytes
    key = bytes.fromhex(key)
    res = pyDes.triple_des(key=key, mode=pyDes.CBC, IV=iv).encrypt(plaintext, padmode=2)
    print(f'encrypted text: {res.hex().upper()}')
# Make the decryption
decrypted_plaintext = pyDes.triple_des(key, pyDes.CBC, IV=iv).decrypt(res, padmode=2).hex().upper()
print(f'decrypted plaintext: {decrypted_plaintext}')

original plaintext: 4E3F0E5A000000010018BC
encrypted text: CF5496DC7EAA4A2656F5D960B554AEDD
encrypted text: 603D79FB432EAE63FE2D065B49C52121
encrypted text: 623E1C793DB643EFFBB22C0484654045
encrypted text: AF42174E65F43EBEDF01B382F3C63088
encrypted text: 9974CD4BB7BD3D122A5F706EB03C4138
encrypted text: D2DBDE6457BA911C32AD0CB0AAB139B9
encrypted text: 8C0FF801F874B29E33D292621C789964
encrypted text: 0EBDF014547B22BB24E576184BFFC3E4
encrypted text: 9C2A2789FD49951207A2FE9E3DFCF1AE
encrypted text: 907280CA7A46CE09BEF86E5D918D9429
decrypted plaintext: 4E3F0E5A000000010018BC


In [ ]:
# Декодировка

encrypted_text = '603D79FB432EAE63F082E734F3B46085'
encrypted_text = bytes.fromhex(encrypted_text)

key_list = [
    'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1',
    '26B60249EC3D7CCE4AADE73F0D803E470AEA1E2C0279AD5D',
    '9B68C23B919D620B9F8627ECFC8E0BD824197152B4C9CB0E',
    'D04C6E02F201B9E057F6DC45ADD217D7C07ABA5CED76ABF4',
    '67A1C10D23239E190495643C3A233B76D1CE047A9D7D5602',
    '0104BA83E6B3B64FB0207B309B6E40C3EBB7E10B0D751920',
    'DCF410DA0BC4E552F11EFF1FA30B193A5B0FACA7061C310E',
    'CB43F7734A0DFB4AA332E690CE0D31F31CB9A6024804DAF2',
    'A4CDDF91ADD34649AD1956FE90333EF5C3012EBF25D7EB53',
    'FEEA7CA8ECE6DAF856EEE86785F7DFE5AE3FF8978E4D1F5B',
]

for key in key_list:
    # Convert key to bytes
    iv = bytes.fromhex('0000000000000000')
    key = bytes.fromhex(key)
    decrypted_plaintext = pyDes.triple_des(key, pyDes.CBC, IV=iv).decrypt(encrypted_text, padmode=1).hex().upper()
    print(f'decrypted plaintext: {decrypted_plaintext}')

### Send SMS via SMPP

In [363]:
import smpplib
import smpplib.gsm
client = None
parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts(u'Hello world!')
# parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts("""Hello world!""")

client = smpplib.client.Client('172.18.11.20', 2770) # Prod
# client = smpplib.client.Client('172.18.11.90', 2770) # Lab

client.connect()
client.bind_transceiver(system_id='test', password='test') # Prod
# client.bind_transceiver(system_id='test', password='ChangeMe') # Lab

for part in parts:
    pdu = client.send_message(
        source_addr_ton=smpplib.consts.SMPP_TON_INTL,
#         source_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure it is a byte string, not unicode:
#         source_addr='16197688802',
        source_addr='972559300037',
#         source_addr='IsraelPostIsraelPost',
        dest_addr_ton=smpplib.consts.SMPP_TON_INTL,
        dest_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure thease two params are byte strings, not unicode:
#         destination_addr='16474936003',
        destination_addr='79219832572',
        short_message=part,
        data_coding=encoding_flag,
        esm_class=msg_type_flag,
        registered_delivery=True,)
    print(f'PDU SQN: {pdu.sequence}')
client.unbind()
client.disconnect()

PDU SQN: 2


<smpplib.client.Client object at 0x00000245F8CB1BE0> is disconnecting in the bound state


In [354]:
parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts("""00-Hello world! 01-Hello world! 02-Hello world! 03-Hello world! 04-Hello world! 05-Hello world! 06-Hello world! 07-Hello world! 08-Hello world! 09-Hello world! 10-Hello world! 11-Hello world! 12-Hello world! 13-Hello world! 14-Hello world! 15-Hello world!""")
msg_type_flag

64

## Testing

In [ ]:
import requests
ip_addr = '192.168.1.44'
port = 80
url_01 = f'http://{ip_addr}:{port}/climate;sensor=1&readattempt=10&temperature=25.0&humidity=35.0'
url_02 = f'http://{ip_addr}:{port}/climate;sensor=4&readattempt=10&temperature=25.0&humidity=35.0'
url_03 = f'http://{ip_addr}:{port}/gas;sensor=2&sensorValue=666'
url_04 = f'http://{ip_addr}:{port}/motion;sensor=3'
for url in [url_01, url_02, url_03, url_04]:
    r = requests.get(url)
    print(r.status_code)
    print(r.text.strip())

In [369]:
# Data

sdrRate = 1.441
charge = 0.00122
chargedUnits = 481280
unit = 1024 * 1014
(charge * sdrRate / chargedUnits) * unit
# 0.0349

0.003792834638297872

In [ ]:
# Sms

sdrRate = 1.37522
charge = 0.00122
chargedUnits = 1
unit = 1
(charge * sdrRate / chargedUnits) * unit

In [372]:
# Voice

sdrRate = 1.441
charge = 68.75584
chargedUnits = 655
unit = 60
(charge * sdrRate / chargedUnits) * unit

9.075770880000002

In [206]:
# Data Speed calculation

data_size = 481280
time = 43
(data_size / time) / (1024 * 1024)

0.23255813953488372

In [373]:
68.75584 * 1.441

99.07716544000002

In [37]:
(dt.datetime.now() - dt.timedelta(weeks=10)).strftime("%Y-%m-%d")

'2020-11-07'

### Developments

In [31]:
import requests
ip_addr = '192.168.1.52'
ip_addr = '85.21.168.42'
port = 80
url = f'http://{ip_addr}:{port}/connected;sensor=1&ip=192.168.1.52&ver=210103'
r = requests.get(url)
print(r.status_code, r.text.strip())
# r = requests.post(url, data="firstname=Pavel&lastname=Balobin")
# print(r.status_code, r.text.strip())

200 OK


In [421]:
75000+4758.97

79758.97

In [355]:
import requests
r = requests.get("https://localhost:5684/", verify=join(downloads, "ca-public-key.pem"))
print(r.status_code, r.text.strip())

SSLError: HTTPSConnectionPool(host='localhost', port=5684): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')],)",),))

In [419]:
2799 / 1000  # 2.799
1999 / 700  # 2.855

2.855714285714286

In [420]:
300 * 2.799

839.6999999999999

In [410]:
dir(res1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_resource_id',
 'resource_id']

In [437]:
# Soft profiles:
# 45201(Partner)-20 pcs
425019613998538-425019613998557

# 26006(P4)-70 pcs
260060149991555-260060149991624

# 51503(SMART)-10pcs
515030191000763-515030191000772

# 26003(Centertel)-100 pcs
260036610100114-260036610100213

260036610100213

In [438]:
425019613998531-425019613998532

-1

In [55]:
%reset -f